This notebook is to create a network including the nodes and links from Gijs' "werkgevers aansprakelijkheid" network.

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
import os
import sys
sys.path.insert(0, os.path.abspath('..'))
import  parser.matcher as matcher

In [3]:
sparql = SPARQLWrapper("http://localhost:9999/blazegraph/namespace/hogeraad/sparql")

In [4]:
import pandas as pd
filepath_csv = '/media/sf_VBox_Shared/CaseLaw/graphs/test3 wg aanspr.csv'
links_wgaans = pd.read_csv(filepath_csv, sep=';', header=0)
links_wgaans.columns = ['id', 'reference']
links_wgaans = links_wgaans.drop_duplicates()

In [5]:
links_wgaans = links_wgaans.applymap(lambda ecli: 'http://deeplink.rechtspraak.nl/uitspraak?id='+ecli)

In [6]:
ids_wgaans = list(pd.concat([links_wgaans['id'], links_wgaans['reference']]).unique())

In [7]:
ids_wgaans[0]

'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2012:BV1295'

In [8]:
node_list = []
for ecli in ids_wgaans:
    querystring = '''
            prefix dcterm: <http://purl.org/dc/terms/>
            select ?type ?id ?to ?title ?creator ?date ?subject ?abstract ?hasVersion
            where {
                {
                BIND("node" AS ?type).
                ?id dcterm:type	<http://psi.rechtspraak.nl/uitspraak>.
                filter (?id = <''' + ecli+'''>).
                optional { ?id dcterm:creator ?creator}.
                   optional { ?id dcterm:abstract ?abstract}.
                   optional { ?id dcterm:subject ?subject}.
                   optional { ?id dcterm:date ?date}.
                   optional { ?id dcterm:title ?title}
                }
                
                union
                {
                BIND("vindplaats" AS ?type).
                ?id dcterm:hasVersion ?hasVersion
                filter (?id = <''' + ecli+'''>).
                }
              }
            '''
    #print(querystring)
    sparql.setQuery(querystring)
    sparql.setReturnFormat(JSON)
    ret = sparql.query()
    node_list.append(ret.convert())

In [9]:
nodes = [res for node in node_list for res in node['results']['bindings'] if res['type']['value']=='node']

In [10]:
len(nodes)

153

In [11]:
nodes[0]

{'abstract': {'type': 'literal',
  'value': 'Uitleg dekkingsomvang AVB-polis. Aansprakelijkheid werkgever bij verkeersongeval werknemer op grond van art. 7:611 BW (verzekeringsplicht niet nagekomen). Is schade werknemer letselschade in zin van polis? Functie AVB-polis en daarop gebaseerde verwachtingen rechtvaardigen ruime uitleg dekkingsomvang; dekking tegen aansprakelijkheid voor letselschade omvat in beginsel ook dekking tegen op art. 7:611 gebaseerde aansprakelijkheid.'},
 'creator': {'type': 'uri',
  'value': 'http://standaarden.overheid.nl/owms/terms/Hoge_Raad_der_Nederlanden'},
 'date': {'datatype': 'http://www.w3.org/2001/XMLSchema#date',
  'type': 'literal',
  'value': '2012-03-30'},
 'id': {'type': 'uri',
  'value': 'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2012:BV1295'},
 'subject': {'type': 'uri',
  'value': 'http://psi.rechtspraak.nl/rechtsgebied#civielRecht'},
 'title': {'type': 'literal',
  'value': 'ECLI:NL:HR:2012:BV1295 Hoge Raad , 30-03-2012 / 11/01010'

In [12]:
vindplaatsen = [res for node in node_list for res in node['results']['bindings'] if res['type']['value']=='vindplaats']

In [13]:
len(vindplaatsen)

812

In [14]:
def nodes_to_json(nodes_in, variables):
    id_to_node = {}
    for d in nodes_in:
        id0 = d['id']['value']
        dout = {}
        for var in variables:
            if var in d.keys():
                dout[var] = d[var]['value']#d.get(var, {'value':''})['value']
            else:
                dout[var] = ''
        id_to_node[id0] = dout
            
    return id_to_node

In [15]:
variables = [x for x in node_list[0]['head']['vars'] if x not in ['type', 'from', 'to']]
id_to_node = nodes_to_json(nodes, variables)

In [16]:
variables

['id', 'title', 'creator', 'date', 'subject', 'abstract', 'hasVersion']

In [17]:
node_example = id_to_node['http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2012:BV1295']


In [18]:
empty_node = {var: '' for var in variables if var != 'id'}

nodes_json = []
for ecli in ids_wgaans:
    id0 = ecli
    nodes_json.append(id_to_node.get(id0, {'id':id0, **empty_node}))

In [19]:
print([node for node in nodes_json if node['id']=='http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2012:BV1295'])

[{'id': 'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2012:BV1295', 'subject': 'http://psi.rechtspraak.nl/rechtsgebied#civielRecht', 'creator': 'http://standaarden.overheid.nl/owms/terms/Hoge_Raad_der_Nederlanden', 'hasVersion': '', 'title': 'ECLI:NL:HR:2012:BV1295 Hoge Raad , 30-03-2012 / 11/01010', 'abstract': 'Uitleg dekkingsomvang AVB-polis. Aansprakelijkheid werkgever bij verkeersongeval werknemer op grond van art. 7:611 BW (verzekeringsplicht niet nagekomen). Is schade werknemer letselschade in zin van polis? Functie AVB-polis en daarop gebaseerde verwachtingen rechtvaardigen ruime uitleg dekkingsomvang; dekking tegen aansprakelijkheid voor letselschade omvat in beginsel ook dekking tegen op art. 7:611 gebaseerde aansprakelijkheid.', 'date': '2012-03-30'}]


In [20]:
print(len(ids_wgaans), len(nodes_json), len(id_to_node))

156 156 153


In [21]:
# Extra preprocessing step: add articles:
for node in nodes_json:
    articles = matcher.get_articles(node['abstract'])
    node['articles'] = {art+' '+book:cnt for (art,book), cnt in articles.items()}

In [22]:
# Add the number and nr of annotations of vindplaatsen
count_version = {}
count_annotation = {}
for item in vindplaatsen:
    id0 = item['id']['value']
    val = item['hasVersion']['value']
    count_version[id0] = count_version.get(id0 , 0) + 1
    if val.lower().find('met annotatie') >= 0:
        count_annotation[id0] = count_annotation.get(id0 , 0) + 1

In [23]:
for node in nodes_json:
    node['count_version'] = count_version.get(node['id'], 0)
    node['count_annotation'] = count_annotation.get(node['id'], 0)

In [24]:
links = [{'source': row['id'], 'target': row['reference'], 'id': row['id']+'_'+row['reference']} 
         for ind, row in links_wgaans.iterrows()]

In [25]:
links[0]

{'id': 'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2012:BV1295_http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2011:BR5215',
 'source': 'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2012:BV1295',
 'target': 'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2011:BR5215'}

In [27]:
print(len(links), len(links_wgaans))

177 177


In [28]:
nodes_json[6]

{'abstract': 'Arbeidsrecht. Werkgeversaansprakelijkheid; bij eenzijdig verkeersongeval geleden letselschade door fietsende werknemer (thuiszorger) tijdens uitoefening van haar werkzaamheden; zorgplicht van werkgever ex art. 7:658 BW en aanvullende verzekeringsplicht op grond van art. 7:611 BW; behoorlijke verzekering van aan verkeer op de openbare weg deelnemende werknemers.',
 'articles': {'7:611 bw': 1, '7:658 bw': 1},
 'count_annotation': 2,
 'count_version': 9,
 'creator': 'http://standaarden.overheid.nl/owms/terms/Hoge_Raad_der_Nederlanden',
 'date': '2008-12-12',
 'hasVersion': '',
 'id': 'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2008:BD3129',
 'subject': 'http://psi.rechtspraak.nl/rechtsgebied#civielRecht',
 'title': 'ECLI:NL:HR:2008:BD3129 Hoge Raad , 12-12-2008 / C07/121HR'}

In [29]:
import json

filename = '/media/sf_VBox_Shared/CaseLaw/graphs/json_sigma/wgaans.json'

with open(filename, 'w') as outfile:
    json.dump({'nodes': nodes_json, 'edges': links}, fp=outfile)

In [30]:
import json

filename = '/media/sf_VBox_Shared/CaseLaw/graphs/d3_wgaans.json'

with open(filename, 'w') as outfile:
    json.dump({'nodes': nodes_json, 'links': links}, fp=outfile)